# Paper Notebook: Robustness & Peak-Critical Evaluation

This notebook is **evaluation-only** (publication-focused).
It loads the pre-built strict dataset artifact and trained model weights, then produces:

- Table A: clean MAE/RMSE (12/24/48/72h)
- Table B: sensor outage robustness (overall + masked-only)
- Table C: peak-only MAE/RMSE
- Figure: masked-only MAE@72 vs outage rate

Training is performed in the separate *STGCN training notebook* and saved to disk.

In [2]:
import os, json, math, time, random
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if DEVICE == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

Device: cuda
GPU: Quadro P5000


In [4]:
DATA_PATH = Path("artifacts/pems_graph_dataset_strict.npz")
assert DATA_PATH.exists(), (
    f"Missing {DATA_PATH}.\n"
    "Fix:\n"
    "1) Run the dataset-build part of your STGCN notebook (it creates this file), OR\n"
    "2) Make sure you copied the entire 'artifacts/' folder into this project directory."
)

data = np.load(DATA_PATH, allow_pickle=True)
print("Loaded:", DATA_PATH)
print("Keys:", list(data.keys()))

X_raw = data["X"].astype(np.float32)   # (T, N, F) raw flow/speed + possibly other raw features
Y_raw = data["Y"].astype(np.float32)   # (T, N) raw flow target
A     = data["A"].astype(np.float32)   # (N, N) adjacency

stations   = data["stations"]          # (N,)
timestamps = data["timestamps"]        # (T,)

flow_mean  = data["flow_mean"].astype(np.float32)   # (N,)
flow_std   = data["flow_std"].astype(np.float32)    # (N,)
speed_mean = data["speed_mean"].astype(np.float32)  # (N,)
speed_std  = data["speed_std"].astype(np.float32)   # (N,)

IN_LEN  = int(data["in_len"])   # your L
OUT_LEN = int(data["out_len"])  # your H

train_starts = data["train_starts"].astype(np.int64)
val_starts   = data["val_starts"].astype(np.int64)
test_starts  = data["test_starts"].astype(np.int64)

T, N, F_in = X_raw.shape
print("Shapes:")
print("  X_raw:", X_raw.shape, "(T,N,F)")
print("  Y_raw:", Y_raw.shape, "(T,N)")
print("  A    :", A.shape, "(N,N)")
print("  IN_LEN/OUT_LEN:", IN_LEN, OUT_LEN)

assert T >= (IN_LEN + OUT_LEN), f"T={T} is too small for IN_LEN+OUT_LEN={IN_LEN+OUT_LEN}."

Loaded: artifacts/pems_graph_dataset_strict.npz
Keys: ['X', 'Y', 'A', 'stations', 'timestamps', 'train_starts', 'val_starts', 'test_starts', 'in_len', 'out_len', 'flow_mean', 'flow_std', 'speed_mean', 'speed_std']
Shapes:
  X_raw: (2208, 1821, 6) (T,N,F)
  Y_raw: (2208, 1821) (T,N)
  A    : (1821, 1821) (N,N)
  IN_LEN/OUT_LEN: 24 72


/tmp/ipykernel_526/2834048384.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  IN_LEN  = int(data["in_len"])   # your L
/tmp/ipykernel_526/2834048384.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  OUT_LEN = int(data["out_len"])  # your H


In [5]:
def time_encoding(dt_index: pd.DatetimeIndex) -> np.ndarray:
    hours = dt_index.hour.values
    dow   = dt_index.dayofweek.values
    hour_sin = np.sin(2*np.pi*hours/24.0)
    hour_cos = np.cos(2*np.pi*hours/24.0)
    dow_sin  = np.sin(2*np.pi*dow/7.0)
    dow_cos  = np.cos(2*np.pi*dow/7.0)
    return np.stack([hour_sin, hour_cos, dow_sin, dow_cos], axis=1).astype(np.float32)

dt_idx = pd.to_datetime(timestamps)
TF_all = time_encoding(dt_idx)         # (T,4)

# Scale inputs
X_scaled = X_raw.copy()
# Assumption consistent with your STGCN notebook: channel 0=flow, channel 1=speed
X_scaled[:, :, 0] = (X_scaled[:, :, 0] - flow_mean[None, :]) / (flow_std[None, :] + 1e-6)
X_scaled[:, :, 1] = (X_scaled[:, :, 1] - speed_mean[None, :]) / (speed_std[None, :] + 1e-6)

# Scale target (flow)
Y_scaled = (Y_raw - flow_mean[None, :]) / (flow_std[None, :] + 1e-6)

# For fast slicing: (F,N,T)
X_fnt = np.transpose(X_scaled, (2, 1, 0)).copy()  # (F,N,T)

print("X_fnt:", X_fnt.shape, "| Y_scaled:", Y_scaled.shape, "| TF_all:", TF_all.shape)

X_fnt: (6, 1821, 2208) | Y_scaled: (2208, 1821) | TF_all: (2208, 4)
